# Symbolic generator of the full density matrices

## Context

In this notebook the full density matrices for the three-box problem are symbolically computed using the *itertools.combinations* function based on the formula

$$
\rho_A = \frac{1}{\binom{2N}{N}} \sum_{\tau \in J_N}  | \Psi_A^\tau \rangle \langle \Psi_A^\tau | ,
$$

where $J_N$ is the set of all $N$-combinations of $J=\{1,\dots,2N\}$ and $\Psi_A^\tau$ is the state for which photons with index $j\in \tau$ are in state $H$ and the rest in state $V$. The full density matrix $\rho_B$ is similarly defined, and $\rho_C$ is given by

$$
\rho_C = \underbrace{\left(\frac{1}{2} \mathrm{I}\right) \otimes \dots \otimes \left(\frac{1}{2} \mathrm{I}\right)}_{2N\text{ times}} = \frac{1}{2^{2N}} \mathrm{I}^{\otimes 2N},
$$

where $\mathrm I$ is the identity operator in the single-photon polarization state space $\mathbb C ^2$.

## Outputs

- Symbolically computed full density matrices of boxes A, B and C

In [1]:
import pickle
from itertools import combinations
from sympy.matrices import Matrix, eye
from sympy import sqrt, I
from sympy.physics.quantum import TensorProduct
from sympy.functions.combinatorial.factorials import binomial

In [17]:
# simulation parameters

n = 4 # half-number of photons
N = 2*n # total number of photons
d = 2**N # hilbert space dimension
C = binomial(N,n) # number of photon combinations
S = ["V"]*2*n

In [18]:
# computing all photon combinations

comb_list = [] # list of all photon combinations

for c in combinations(range(N),n):
    s = S.copy()
    for i in c: s[i] = 'H'
    comb_list.append(s)
    
print(comb_list[1]) # instance

['H', 'H', 'H', 'V', 'H', 'V', 'V', 'V']


In [19]:
# defining photon state vectors

H = Matrix([1,0]); V = Matrix([0,1]) # HV states
L = Matrix([1,I])/sqrt(2); R = Matrix([1,-I])/sqrt(2) # LR states

get_vecA = {"H" : H, "V" : V}
get_vecB = {"H" : L, "V" : R}

In [20]:
# building the full density matrices

rhoA = Matrix([[0]*d]*d)
rhoB = Matrix([[0]*d]*d)

for c in comb_list:
    # print(*c) # uncomment to see all combinations
    vec = get_vecA[c[0]]
    
    [vec := TensorProduct(vec, get_vecA[l]) for l in c[1:]]
    rhoA += vec.multiply(vec.T.conjugate())
    
    vec = get_vecB[c[0]]
    
    [vec := TensorProduct(vec, get_vecB[l]) for l in c[1:]]
    rhoB += vec.multiply(vec.T.conjugate())
    
rhoA = rhoA/C
rhoB = rhoB/C

rhoC = eye(d)/d # maximum entropy density matrix

In [21]:
# save the matrices in binary files

with open("full_density_matrices/rhoA_N{:d}_sym.bin".format(N), "wb") as outf:
    pickle.dump(rhoA, outf)
    
with open("full_density_matrices/rhoB_N{:d}_sym.bin".format(N), "wb") as outf:
    pickle.dump(rhoB, outf)
    
with open("full_density_matrices/rhoC_N{:d}_sym.bin".format(N), "wb") as outf:
    pickle.dump(rhoC, outf)